In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys

proj_root = os.path.dirname(os.path.abspath("."))
# print(proj_root)
sys.path.append(proj_root)

from minatar_dqn.utils import my_logging
from minatar_dqn import my_dqn, models
from experiments.experiment_utils import seed_everything
from minatar_dqn.replay_buffer import ReplayBuffer
from minatar_dqn.redo import ReDo, apply_redo_parametrization, register_redo_hooks

import random
import torch
import torch.nn as nn
import time


c:\Users\Chainsword\anaconda3\envs\general\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
net = models.Conv_QNET(in_features=(4, 10, 10), in_channels=4, num_actions=6)

net = apply_redo_parametrization(net, tau=0.1)

D = 10
for i in range(100):
    x = torch.rand((32, 4, D, D))
    net(x)
    if i % 10 == 0:
        # print(i, net.get_dormant_ratios())
        # print(i, net.get_dormant_scores())
        scores = net.get_dormant_scores()
        ratios, mus = net.get_dormant_ratios()
        break

Hooking Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1)) -> Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
Hooking Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1)) -> Linear(in_features=576, out_features=64, bias=True)
Hooking Linear(in_features=576, out_features=64, bias=True) -> Linear(in_features=64, out_features=6, bias=True)


In [47]:
net

Conv_QNET(
  (features): Sequential(
    (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=576, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [43]:
ratios

[0.1875, 0.5, 0.5625]

In [46]:
scores

[tensor([8.8169e-02, 2.5198e-01, 1.6260e-03, 8.8824e-02, 1.7560e-03, 1.2273e-01,
         5.8098e-02, 2.1921e-04, 3.1060e-02, 5.7404e-02, 1.3044e-01, 2.3220e-02,
         1.6962e-02, 1.0153e-01, 1.4443e-02, 1.1541e-02]),
 tensor([4.3086e-02, 3.0800e-01, 1.6793e-04, 3.2917e-04, 4.6855e-02, 1.0855e-02,
         4.1735e-01, 1.0977e-03, 2.9207e-04, 1.1826e-01, 0.0000e+00, 4.0282e-03,
         3.9717e-02, 0.0000e+00, 5.4572e-04, 9.4183e-03]),
 tensor([0.0000e+00, 4.9115e-03, 9.8421e-04, 7.9603e-04, 9.9051e-03, 3.8353e-03,
         0.0000e+00, 0.0000e+00, 5.9351e-02, 0.0000e+00, 6.6987e-02, 2.9111e-04,
         0.0000e+00, 0.0000e+00, 6.2451e-03, 4.2907e-03, 2.2899e-02, 3.7878e-02,
         0.0000e+00, 3.9319e-02, 7.5884e-02, 4.7591e-02, 2.9076e-02, 9.4271e-04,
         1.2377e-02, 3.1884e-02, 5.9845e-03, 1.4825e-03, 0.0000e+00, 3.8298e-02,
         3.1558e-03, 3.8014e-03, 3.1426e-02, 0.0000e+00, 0.0000e+00, 3.0370e-02,
         1.5595e-02, 3.2816e-02, 4.4755e-03, 3.7311e-02, 1.9402e-03, 2.2

In [44]:
for score in scores:
    print(sum(score < 0.1) / len(score)) 

tensor(0.7500)
tensor(0.8125)
tensor(0.9844)


In [45]:
for name, param in net.named_parameters():
    print(name)
    print(param.data)


features.0.weight
tensor([[[[-0.0456,  0.1411,  0.0173],
          [ 0.0241, -0.0145, -0.0077],
          [-0.1380,  0.0896,  0.0243]],

         [[-0.0843,  0.1087,  0.0161],
          [-0.0115, -0.1362, -0.1302],
          [ 0.0259, -0.0401, -0.1019]],

         [[-0.0804,  0.1314,  0.0323],
          [ 0.0432, -0.1641, -0.0161],
          [ 0.0352,  0.1449, -0.1098]],

         [[ 0.1313, -0.0291,  0.0401],
          [ 0.0472,  0.1426,  0.1622],
          [ 0.0774, -0.0140,  0.1374]]],


        [[[ 0.0765,  0.1164, -0.0548],
          [ 0.1150,  0.0976,  0.0018],
          [ 0.1086,  0.1187,  0.0053]],

         [[-0.0345,  0.0479,  0.1652],
          [-0.0421, -0.0643, -0.0744],
          [ 0.0328,  0.1477, -0.0187]],

         [[-0.0379, -0.0778, -0.1073],
          [ 0.1565,  0.1510,  0.1203],
          [ 0.0843,  0.1094,  0.1027]],

         [[ 0.0816,  0.1292,  0.0652],
          [-0.0877,  0.0835, -0.0174],
          [ 0.1217, -0.0393,  0.1368]]],


        [[[-0.0636, -0.048

In [39]:
for name, param in net.named_parameters():
    if "weight" in name:
        num_neurons = param.shape[0]
        print(f"{name} has {num_neurons} neurons")

features.0.weight has 16 neurons
features.2.weight has 16 neurons
fc.0.weight has 64 neurons
fc.2.weight has 6 neurons
